In [1]:
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt
import segmentation_models as sm
import keras.backend as K
import tensorflow as tf

from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dropout, Conv2DTranspose, concatenate, UpSampling2D
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras import Model
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from keras.metrics import MeanIoU

Segmentation Models: using `keras` framework.


In [2]:
sm.set_framework('tf.keras')

In [3]:
TRAIN_PICS_DIRECTORY = '/kaggle/input/airbus-ship-detection/train_v2'
CSV_SHIP_PATH = "/kaggle/input/filtered-ships-airbus/train_filtered_ships.csv"

In [4]:
def mask_converter(values):
    mask = np.zeros((768*768,), dtype=float)        #create empty one-dimentional vector with zeros
    if values != "none":
        values = values.strip().split()
        start_points = values[0::2]               #separate values
        lengths = values[1::2]
        for st_p, l in zip(start_points, lengths):     #fill mask with ones according to the EncodedPixels colomn
            st_p, l = int(st_p)-1, int(l)
            ones = np.ones(l, dtype=int) 
            mask[int(st_p):int(st_p)+int(l)] = ones
    return np.transpose(mask.reshape((768, 768, 1)), axes=(1, 0, 2))    #rotate image to get correct orientation

def dice_coef(y_true, y_pred, smooth=1):
    intersection = K.sum(y_true * y_pred, axis=[1,2,])
    union = K.sum(y_true, axis=[1,2]) + K.sum(y_pred, axis=[1,2])
    return K.mean( (2. * intersection + smooth) / (union + smooth), axis=0)

def dice_loss(y_true, y_pred):
    return 1-dice_coef(y_true, y_pred)

def iou_coef(y_true, y_pred, smooth=1):
    true = K.flatten(y_true)
    pred = K.flatten(y_pred)
    intersection = K.sum(true*pred)
    union = K.sum(true) + K.sum(pred) - intersection
    return (intersection+smooth)/(union+smooth)

def iou_loss(y_true, y_pred):
    return -iou_coef(y_true, y_pred)
    

# initializing U-net model
def unet_model(input_size=(256,256,3)):
    inputs = Input(input_size)

    c1 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(inputs)
    c1 = Dropout(0.1)(c1)
    c1 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c1)
    p1 = MaxPooling2D((2, 2))(c1)

    c2 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p1)
    c2 = Dropout(0.1)(c2)
    c2 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c2)
    p2 = MaxPooling2D((2, 2))(c2)

    c3 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p2)
    c3 = Dropout(0.2)(c3)
    c3 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c3)
    p3 = MaxPooling2D((2, 2))(c3)

    c4 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p3)
    c4 = Dropout(0.2)(c4)
    c4 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c4)
    p4 = MaxPooling2D(pool_size=(2, 2))(c4)

    c5 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p4)
    c5 = Dropout(0.3)(c5)
    c5 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c5)

    u6 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c5)
    u6 = concatenate([u6, c4])
    c6 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u6)
    c6 = Dropout(0.2)(c6)
    c6 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c6)

    u7 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c6)
    u7 = concatenate([u7, c3])
    c7 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u7)
    c7 = Dropout(0.2)(c7)
    c7 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c7)

    u8 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c7)
    u8 = concatenate([u8, c2])
    c8 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u8)
    c8 = Dropout(0.1)(c8)
    c8 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c8)

    u9 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(c8)
    u9 = concatenate([u9, c1], axis=3)
    c9 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u9)
    c9 = Dropout(0.1)(c9)
    c9 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c9)

    d = Conv2D(1, (1, 1), activation='sigmoid')(c9)
    
    return Model(inputs=[inputs], outputs=[d])


def data_generator(csv_data_file, image_folder_path, batch_size, epochs, preprocessing=None, augmentation=False, aug_cycles=3, aug_batch_size=32):              
    """
    Yields the next data batch.
    """
    labels_file = pd.read_csv(csv_data_file)
    num_images = len(labels_file)
    try:
        preprocess = sm.get_preprocessing(preprocessing)
    except:
        preprocess = None
    for _ in range(epochs):
        for offset in range(0, num_images, batch_size):        
            # Get the samples you'll use in this batch
            batch_images = labels_file['ImageId'][offset:offset+batch_size].values
            batch_masks = labels_file['EncodedPixels'][offset:offset+batch_size].values
            # Initialise X_train and y_train arrays for this batch
            X_train = []
            y_train = []
            # For each example
            for image_filename, mask_encoded in zip(batch_images, batch_masks):          
                # Add example to arrays
                image = cv2.imread(f"{image_folder_path}/{image_filename}")
                mask = mask_converter(mask_encoded)
                image = cv2.resize(image, (256, 256), interpolation=None)
                mask = cv2.resize(mask, (256, 256), interpolation=cv2.INTER_NEAREST)
                X_train.append(image)
                y_train.append(mask)

            X_train = np.array(X_train)
            y_train = np.array(y_train)[..., np.newaxis]

            if augmentation:
                image_gen = ImageDataGenerator(horizontal_flip=True,
                                               vertical_flip=True,
                                               width_shift_range=0.2,
                                               height_shift_range=0.2
                                                          )
                mask_gen = ImageDataGenerator(horizontal_flip=True,
                                              vertical_flip=True,
                                              width_shift_range=0.2,
                                              height_shift_range=0.2
                                                          )
                image_gen.fit(X_train)
                mask_gen.fit(y_train)
                counter = 0
                for X_aug_train, y_aug_train in zip(image_gen.flow(X_train, batch_size=aug_batch_size, seed=42), mask_gen.flow(y_train, batch_size=aug_batch_size, seed=42)):
                    if counter == aug_cycles:
                        counter = 0
                        break
                    counter += 1
                    yield (X_aug_train, y_aug_train) if preprocessing is None else (preprocess(X_aug_train), y_aug_train)
            else:
                yield (X_train, y_train) if preprocessing is None else (preprocess(X_train), y_train)
                # yield the next training batch

In [8]:
dataset_size = len(pd.read_csv(CSV_SHIP_PATH))
dataset_size

42556

In [9]:
BATCH_SIZE = 32
EPOCHS = 5
AUG_CYCLES = 3
AUG_BATCH_SIZE = 16

In [10]:
optimizer = Adam(learning_rate=0.001)

dice_loss = sm.losses.DiceLoss(class_weights=np.array([0.5, 0.5]))
focal_loss = sm.losses.CategoricalFocalLoss()
total_loss = dice_loss + (1*focal_loss)

metrics = [dice_coef, sm.metrics.IOUScore(threshold=0.5), sm.metrics.FScore(threshold=0.5)]

### EficientNet

In [11]:
efficientb5_model = sm.Unet('efficientnetb5', encoder_weights='imagenet', classes=1, activation='sigmoid')

efficientb5_model.compile(optimizer, total_loss, metrics)

In [12]:
train_augmented_generator = data_generator(
    csv_data_file=CSV_SHIP_PATH, 
    image_folder_path=TRAIN_PICS_DIRECTORY, 
    preprocessing='efficientnetb5', 
    batch_size=BATCH_SIZE, 
    epochs=EPOCHS,
    augmentation=True, 
    aug_cycles=AUG_CYCLES,
    aug_batch_size=AUG_BATCH_SIZE
)

In [13]:
efficientb5_model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
stem_conv (Conv2D)              (None, None, None, 4 1296        input_1[0][0]                    
__________________________________________________________________________________________________
stem_bn (BatchNormalization)    (None, None, None, 4 192         stem_conv[0][0]                  
__________________________________________________________________________________________________
stem_activation (Activation)    (None, None, None, 4 0           stem_bn[0][0]                    
______________________________________________________________________________________________

In [14]:
efficientb5_model_history = efficientb5_model.fit(train_augmented_generator, steps_per_epoch=(dataset_size//BATCH_SIZE+1)*AUG_CYCLES*EPOCHS)

19950/19950 [==============================] - 20861s 1s/step - loss: 0.6103 - dice_coef: 0.6028 - iou_score: 0.6540 - f1-score: 0.7853


In [15]:
efficientb5_model.save("/kaggle/working/efficientnetb5/")

/opt/conda/lib/python3.7/site-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


### VGG19

In [ ]:
vgg19_model = sm.Unet('vgg19', encoder_weights='imagenet', classes=1, activation='sigmoid')

vgg19_model.compile(optimizer, total_loss, metrics)

In [ ]:
train_augmented_generator = data_generator(
    csv_data_file=CSV_SHIP_PATH, 
    image_folder_path=TRAIN_PICS_DIRECTORY, 
    preprocessing='vgg19', 
    batch_size=BATCH_SIZE, 
    epochs=EPOCHS,
    augmentation=True, 
    aug_cycles=AUG_CYCLES,
    aug_batch_size=AUG_BATCH_SIZE
)

In [ ]:
vgg_19_history = vgg19_model.fit(train_augmented_generator, steps_per_epoch=(dataset_size//BATCH_SIZE+1)*AUG_CYCLES*EPOCHS)

In [ ]:
vgg19_model.save("/kaggle/working/vgg19_10/")

## ResNet34

In [ ]:
resnet34_model = sm.Unet('resnet34', encoder_weights='imagenet', classes=1, activation='sigmoid')

resnet34_model.compile(optimizer, total_loss, metrics)

In [ ]:
resnet34_model.summary()

In [ ]:
train_augmented_generator = data_generator(
    csv_data_file=CSV_SHIP_PATH, 
    image_folder_path=TRAIN_PICS_DIRECTORY, 
    preprocessing='resnet34', 
    epochs=EPOCHS, 
    batch_size=BATCH_SIZE, 
    augmentation=True, 
    aug_batch_size=32
)

In [ ]:
resnet34_history = resnet34_model.fit(train_augmented_generator, steps_per_epoch=(dataset_size//BATCH_SIZE+1)*3*EPOCHS)

In [ ]:
resnet34_model.save("/kaggle/working/resnet34/")

### Test

In [ ]:
TEST_PICS_DIRECTORY = '/kaggle/input/airbus-ship-detection/test_v2'

In [ ]:
test_generator = data_generator(csv_data_file=CSV_SHIP_PATH, image_folder_path=TRAIN_PICS_DIRECTORY, epochs=1, batch_size=32, augmentation=False)

In [ ]:
X_batch, y_batch = next(test_generator)

In [ ]:
preprocess = sm.get_preprocessing('vgg19')

In [ ]:
y_pred = (vgg19_model.predict(preprocess(X_batch)) > 0.25).astype(np.uint8)

In [ ]:
for image in y_batch:
    plt.figure(figsize=(5,5))
    plt.imshow(image)

In [ ]:
for image in y_pred:
    plt.figure(figsize=(5,5))
    plt.imshow(image)